In [858]:
import json
import sys, os
import glob
import logging
logging.getLogger().setLevel(logging.CRITICAL)

from requests import Session, Request
from datetime import datetime, date, time, timedelta, timezone
from posixpath import join as urljoin
import pandas as pd
import io
from blume import client, station, measurements
from blume.station import Station
from sensemapi import client as sense_client
import numpy as np
#import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing, svm 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

import pytz
cet = pytz.timezone('CET')

# set the graphs to show in the jupyter notebook
%matplotlib inline

# set seaborn style to white
#sns.set_style("white")

from IPython.display import display, HTML

In [859]:
logging.getLogger().setLevel(logging.INFO)

In [860]:
def load_data(phenomenon, interval):
    filename = "ressources/{}_Measurments_Berlin_{}.cvs".format(phenomenon, interval)
    try:
        data = pd.read_csv(filename, index_col=0, parse_dates=True)
        data.sort_index(inplace=True)
    except FileNotFoundError:
        logging.warning("Could not find File {}, Returning Empty Dataframe instead".format(filename))                
        data = pd.DataFrame()
    return data

In [861]:
def investigate_stats(df):
    total = df.shape[0]*df.shape[1]
    notnull = df.notnull().sum().sum()
    isnull = df.isnull().sum().sum()
    print("General Shape (Hours x Sensors) = {}".format(df.shape))
    print("Non-NaN Values = {} from {} equals {}%".format(notnull, total, round((notnull/total)*100)))
    print("NaN Values = {} from {} equals {}%".format(isnull, total, round((isnull/total)*100)))

In [862]:
def ind_sensor_lookup(df, precision=70):
    total = df.shape[0]
    threshold = (total/100)*precision
    good_sensors = (df.notnull().sum().sort_values(ascending=False) > 1000).sum()
    print("Sensors that measured more than {}% of the Time according to the Conditional Remove = {} from {}".format(precision, good_sensors, df.shape[1]))

## Konzept/Plan
1. Lade Daten und bereinige diese von eventuel übrig gebliebenen Fehlerhaften Columns/Rows
2. Kürze den Titel der PM10/Feuchte/Temperatur Titel auf den Uniqen SenseBox-Teil
3. Merge Join des Datensatzes auf Rows
4. Merge Join des Datensatzes auf Columns
5. Merge Join des Datensatzes auf jeden einzelnen Messwert
6. Setze Treshold für Temp/Feuchte und wende diesen auf PM10 Messwerte an
7. Untersuche das Ergebniss


Join Strategien der Datensätze:

1. PM10 x Temperatur
2. PM10 x Feuchte
3. PM10 x Temperatur x Feuchte

Threshold Strategien der Datensätze:

1. Untersuche welche Threshold Höhe welche Auswirkungen auf die Daten hat
2. Untersuche, wieviele Sensoren zuverlässig und genügend Messwerte für den Betrachtungszeitraum liefern
3. Setze Limit für die minimale Messabdeckung eines Sensors im Betrachtungszeitraum
4. Finalen Datensatz speichern

Load Datasets and drop possible duplicates immediately

In [863]:
df_osem_pm10 = load_data("PM10", "Hours")
df_osem_pm10 = df_osem_pm10.loc[~df_osem_pm10.index.duplicated(keep='first')]
df_osem_feuchte = load_data("rel. Luftfeuchte", "Hours")
df_osem_feuchte = df_osem_feuchte.loc[~df_osem_feuchte.index.duplicated(keep='first')]
df_osem_temp = load_data("Temperatur", "Hours")
df_osem_temp = df_osem_temp.loc[~df_osem_temp.index.duplicated(keep='first')]

In [864]:
ind_sensor_lookup(df_osem_pm10)
ind_sensor_lookup(df_osem_feuchte)
ind_sensor_lookup(df_osem_temp)

Sensors that measured more than 70% of the Time according to the Conditional Remove = 81 from 131
Sensors that measured more than 70% of the Time according to the Conditional Remove = 84 from 146
Sensors that measured more than 70% of the Time according to the Conditional Remove = 87 from 149


In [865]:
print("------ PM10 Stats ------")
investigate_stats(df_osem_pm10)
print("------ Feuchte Stats ------")
investigate_stats(df_osem_feuchte)
print("------ Temp Stats ------")
investigate_stats(df_osem_temp)

------ PM10 Stats ------
General Shape (Hours x Sensors) = (1344, 131)
Non-NaN Values = 122445 from 176064 equals 70.0%
NaN Values = 53619 from 176064 equals 30.0%
------ Feuchte Stats ------
General Shape (Hours x Sensors) = (1441, 146)
Non-NaN Values = 132632 from 210386 equals 63.0%
NaN Values = 77754 from 210386 equals 37.0%
------ Temp Stats ------
General Shape (Hours x Sensors) = (1441, 149)
Non-NaN Values = 136312 from 214709 equals 63.0%
NaN Values = 78397 from 214709 equals 37.0%


Kürze den Header aller Sensor-DF auf den Uniqen SenseBox Part

In [866]:
df_osem_pm10.columns = df_osem_pm10.columns.map(lambda x: x[:22])
df_osem_feuchte.columns = df_osem_feuchte.columns.map(lambda x: x[:22])
df_osem_temp.columns = df_osem_temp.columns.map(lambda x: x[:22])

Joine die DFs auf SenseBox-IDs. Es bleiben nur noch Sensoren die in beiden DFs vorhanden sind

In [867]:
inter_feuchteXpm10 = df_osem_feuchte.columns.intersection(df_osem_pm10.columns)
feuchteXpm10 = df_osem_feuchte.loc[:, inter_feuchteXpm10]
pm10Xfeuchte = df_osem_pm10.loc[:, inter_feuchteXpm10]

inter_tempXpm10 = df_osem_temp.columns.intersection(df_osem_pm10.columns)
tempXpm10 = df_osem_temp.loc[:, inter_tempXpm10]
pm10Xtemp = df_osem_pm10.loc[:, inter_tempXpm10]

inter_pm10XfeuchteXtemp = pm10Xfeuchte.columns.intersection(pm10Xtemp.columns)
pm10XfeuchteXtemp = df_osem_pm10.loc[:, inter_pm10XfeuchteXtemp]
feuchteXtempXpm10 = df_osem_feuchte.loc[:, inter_pm10XfeuchteXtemp]
tempXfeuchteXpm10 = df_osem_temp.loc[:, inter_pm10XfeuchteXtemp]

In [868]:
print("------ PM10xFeuchte Stats ------")
investigate_stats(pm10Xfeuchte)
print("------ FeuchteXpm10 Stats ------")
investigate_stats(feuchteXpm10)
print("############################################")
print("------ PM10xTemp Stats ------")
investigate_stats(pm10Xtemp)
print("------ TempXpm10 Stats ------")
investigate_stats(tempXpm10)
print("############################################")
print("------ PM10xTempxFeuchte Stats ------")
investigate_stats(pm10XfeuchteXtemp)
print("------ feuchteXtempXpm10 Stats ------")
investigate_stats(feuchteXtempXpm10)
print("------ tempXfeuchteXpm10 Stats ------")
investigate_stats(tempXfeuchteXpm10)

------ PM10xFeuchte Stats ------
General Shape (Hours x Sensors) = (1344, 114)
Non-NaN Values = 105941 from 153216 equals 69.0%
NaN Values = 47275 from 153216 equals 31.0%
------ FeuchteXpm10 Stats ------
General Shape (Hours x Sensors) = (1441, 114)
Non-NaN Values = 111188 from 164274 equals 68.0%
NaN Values = 53086 from 164274 equals 32.0%
############################################
------ PM10xTemp Stats ------
General Shape (Hours x Sensors) = (1344, 115)
Non-NaN Values = 107279 from 154560 equals 69.0%
NaN Values = 47281 from 154560 equals 31.0%
------ TempXpm10 Stats ------
General Shape (Hours x Sensors) = (1441, 115)
Non-NaN Values = 112623 from 165715 equals 68.0%
NaN Values = 53092 from 165715 equals 32.0%
############################################
------ PM10xTempxFeuchte Stats ------
General Shape (Hours x Sensors) = (1344, 114)
Non-NaN Values = 105941 from 153216 equals 69.0%
NaN Values = 47275 from 153216 equals 31.0%
------ feuchteXtempXpm10 Stats ------
General Shape

Wiederhole den Vorgang, aber jetzt auf die Indexe, also die Zeitachse. Dannach sind nur noch gleiche SenseBoxen die zur gleichen Zeit beides gemessen haben dabei.

In [869]:
inter_feuchteXpm10 = df_osem_feuchte.index.intersection(df_osem_pm10.index)
inter_tempXpm10 = df_osem_temp.index.intersection(df_osem_pm10.index)

feuchteXpm10 = feuchteXpm10.loc[inter_feuchteXpm10]
pm10Xfeuchte = pm10Xfeuchte.loc[inter_feuchteXpm10]

tempXpm10 = tempXpm10.loc[inter_tempXpm10]
pm10Xtemp = pm10Xtemp.loc[inter_tempXpm10]


inter_pm10XfeuchteXtemp = pm10Xfeuchte.index.intersection(pm10Xtemp.index)

pm10XfeuchteXtemp = pm10XfeuchteXtemp.loc[inter_pm10XfeuchteXtemp]
feuchteXtempXpm10 = feuchteXtempXpm10.loc[inter_pm10XfeuchteXtemp]
tempXfeuchteXpm10 = tempXfeuchteXpm10.loc[inter_pm10XfeuchteXtemp]

In [870]:
print("------ PM10xFeuchte Stats ------")
investigate_stats(pm10Xfeuchte)
print("------ FeuchteXpm10 Stats ------")
investigate_stats(feuchteXpm10)
print("############################################")
print("------ PM10xTemp Stats ------")
investigate_stats(pm10Xtemp)
print("------ TempXpm10 Stats ------")
investigate_stats(tempXpm10)
print("############################################")
print("------ PM10xTempxFeuchte Stats ------")
investigate_stats(pm10XfeuchteXtemp)
print("------ feuchteXtempXpm10 Stats ------")
investigate_stats(feuchteXtempXpm10)
print("------ tempXfeuchteXpm10 Stats ------")
investigate_stats(tempXfeuchteXpm10)

------ PM10xFeuchte Stats ------
General Shape (Hours x Sensors) = (1344, 114)
Non-NaN Values = 105941 from 153216 equals 69.0%
NaN Values = 47275 from 153216 equals 31.0%
------ FeuchteXpm10 Stats ------
General Shape (Hours x Sensors) = (1344, 114)
Non-NaN Values = 103563 from 153216 equals 68.0%
NaN Values = 49653 from 153216 equals 32.0%
############################################
------ PM10xTemp Stats ------
General Shape (Hours x Sensors) = (1344, 115)
Non-NaN Values = 107279 from 154560 equals 69.0%
NaN Values = 47281 from 154560 equals 31.0%
------ TempXpm10 Stats ------
General Shape (Hours x Sensors) = (1344, 115)
Non-NaN Values = 104901 from 154560 equals 68.0%
NaN Values = 49659 from 154560 equals 32.0%
############################################
------ PM10xTempxFeuchte Stats ------
General Shape (Hours x Sensors) = (1344, 114)
Non-NaN Values = 105941 from 153216 equals 69.0%
NaN Values = 47275 from 153216 equals 31.0%
------ feuchteXtempXpm10 Stats ------
General Shape

Dot-Operation with Boolean Matrix on isnull Values.
Nur wenn zur gleichen Stunde, zum gleichen Sensor aus beiden Datensätzen (Temp/PM10) ein Wert vorliegt, behalte diesen Wert. Ansonsten setze den Wert als NaN = Ungültig.

In [871]:
feuchteXpm10[feuchteXpm10.isnull() != pm10Xfeuchte.isnull()] = np.nan
pm10Xfeuchte[feuchteXpm10.isnull() != pm10Xfeuchte.isnull()] = np.nan

tempXpm10[tempXpm10.isnull() != pm10Xtemp.isnull()] = np.nan
pm10Xtemp[tempXpm10.isnull() != pm10Xtemp.isnull()] = np.nan

pm10XfeuchteXtemp[(pm10XfeuchteXtemp.isnull() != (tempXfeuchteXpm10.isnull() | feuchteXtempXpm10.isnull()))] = np.nan
tempXfeuchteXpm10[(tempXfeuchteXpm10.isnull() != (pm10XfeuchteXtemp.isnull() | feuchteXtempXpm10.isnull()))] = np.nan
feuchteXtempXpm10[(feuchteXtempXpm10.isnull() != (pm10XfeuchteXtemp.isnull() | tempXfeuchteXpm10.isnull()))] = np.nan


In [872]:
print("------ PM10xFeuchte Stats ------")
investigate_stats(pm10Xfeuchte)
print("------ FeuchteXpm10 Stats ------")
investigate_stats(feuchteXpm10)
print("############################################")
print("------ PM10xTemp Stats ------")
investigate_stats(pm10Xtemp)
print("------ TempXpm10 Stats ------")
investigate_stats(tempXpm10)
print("############################################")
print("------ PM10xTempxFeuchte Stats ------")
investigate_stats(pm10XfeuchteXtemp)
print("------ feuchteXtempXpm10 Stats ------")
investigate_stats(feuchteXtempXpm10)
print("------ tempXfeuchteXpm10 Stats ------")
investigate_stats(tempXfeuchteXpm10)

------ PM10xFeuchte Stats ------
General Shape (Hours x Sensors) = (1344, 114)
Non-NaN Values = 102029 from 153216 equals 67.0%
NaN Values = 51187 from 153216 equals 33.0%
------ FeuchteXpm10 Stats ------
General Shape (Hours x Sensors) = (1344, 114)
Non-NaN Values = 102029 from 153216 equals 67.0%
NaN Values = 51187 from 153216 equals 33.0%
############################################
------ PM10xTemp Stats ------
General Shape (Hours x Sensors) = (1344, 115)
Non-NaN Values = 103366 from 154560 equals 67.0%
NaN Values = 51194 from 154560 equals 33.0%
------ TempXpm10 Stats ------
General Shape (Hours x Sensors) = (1344, 115)
Non-NaN Values = 103366 from 154560 equals 67.0%
NaN Values = 51194 from 154560 equals 33.0%
############################################
------ PM10xTempxFeuchte Stats ------
General Shape (Hours x Sensors) = (1344, 114)
Non-NaN Values = 102029 from 153216 equals 67.0%
NaN Values = 51187 from 153216 equals 33.0%
------ feuchteXtempXpm10 Stats ------
General Shape

Erstelle eine MAtrix, in der nur PM10 Messwerte enthalten sind, bei denen zur gleichen Zeit der andere Sensor (Temp oder Feuchte) einen bestimmten Grenzwert nicht überschritten hat (z.B. max. 70% r.F. oder max. 30 Grad)

Versuche in großem Maßsstab: Berechne, wieviele Sensor-Messwerte rausfallen, wenn wir alle Feuchte < 70 raushauen

In [873]:
#For better Visualisation, cut off Dataset here
test_pm10Xtemp = pm10Xtemp.iloc[:, :]
test_tempXpm10 = tempXpm10.iloc[:, :]

test_pm10Xfeuchte = pm10Xfeuchte.iloc[:, :]
test_feuchteXpm10 = feuchteXpm10.iloc[:, :]
#display(test_feuchte)
#display(test_pm10)

test_pm10XfeuchteXtemp = pm10XfeuchteXtemp.iloc[:, :]
test_feuchteXtempXpm10 = feuchteXtempXpm10.iloc[:, :]
test_tempXfeuchteXpm10 = tempXfeuchteXpm10.iloc[:, :]

In [874]:
#set threshold for Feuchte
temp_thresh = 3
feuchte_tresh = 80

In [887]:
print("------ PM10 Stats cut BELOW <= Temp Threshold {}°C ------".format(temp_thresh))
investigate_stats(test_pm10Xtemp[test_tempXpm10<=temp_thresh])
print("------ PM10 Stats cut ABOVE >  Temp Threshold {}°C ------".format(temp_thresh))
investigate_stats(test_pm10Xtemp[test_tempXpm10>temp_thresh])
print("############################################")
print("------ PM10 Stats cut BELOW <= Feuchte Threshold {}%------".format(feuchte_tresh))
investigate_stats(test_pm10Xfeuchte[test_feuchteXpm10<=feuchte_tresh])
print("------ PM10 Stats cut ABOVE >  Feuchte Threshold {}% ------".format(feuchte_tresh))
investigate_stats(test_pm10Xfeuchte[test_feuchteXpm10>feuchte_tresh])
print("############################################")
print("------ PM10 Stats cut BELOW <= Feuchte Threshold {}% AND <= Temp Threshold {}°C------".format(feuchte_tresh, temp_thresh))
below_mask = (test_feuchteXtempXpm10<=feuchte_tresh) & (test_tempXfeuchteXpm10<=temp_thresh)
investigate_stats(test_pm10XfeuchteXtemp[below_mask])
print("------ PM10 Stats cut ABOVE > Feuchte Threshold {}% AND > Temp Threshold {}°C------".format(feuchte_tresh, temp_thresh))
above_mask = (test_feuchteXtempXpm10>feuchte_tresh) & (test_tempXfeuchteXpm10>temp_thresh)
investigate_stats(test_pm10XfeuchteXtemp[above_mask])


------ PM10 Stats cut BELOW <= Temp Threshold 3°C ------
General Shape (Hours x Sensors) = (1344, 115)
Non-NaN Values = 15183 from 154560 equals 10.0%
NaN Values = 139377 from 154560 equals 90.0%
------ PM10 Stats cut ABOVE >  Temp Threshold 3°C ------
General Shape (Hours x Sensors) = (1344, 115)
Non-NaN Values = 88183 from 154560 equals 57.0%
NaN Values = 66377 from 154560 equals 43.0%
############################################
------ PM10 Stats cut BELOW <= Feuchte Threshold 80%------
General Shape (Hours x Sensors) = (1344, 114)
Non-NaN Values = 30441 from 153216 equals 20.0%
NaN Values = 122775 from 153216 equals 80.0%
------ PM10 Stats cut ABOVE >  Feuchte Threshold 80% ------
General Shape (Hours x Sensors) = (1344, 114)
Non-NaN Values = 71588 from 153216 equals 47.0%
NaN Values = 81628 from 153216 equals 53.0%
############################################
------ PM10 Stats cut BELOW <= Feuchte Threshold 80% AND <= Temp Threshold 3°C------
General Shape (Hours x Sensors) = (134

In [888]:
print("------ PM10 Stats cut BELOW <= Temp Threshold {}°C ------".format(temp_thresh))
ind_sensor_lookup(test_pm10Xtemp[test_tempXpm10<=temp_thresh])
print("------ PM10 Stats cut ABOVE >  Temp Threshold {}°C ------".format(temp_thresh))
ind_sensor_lookup(test_pm10Xtemp[test_tempXpm10>temp_thresh])
print("############################################")
print("------ PM10 Stats cut BELOW <= Feuchte Threshold {}%------".format(feuchte_tresh))
ind_sensor_lookup(test_pm10Xfeuchte[test_feuchteXpm10<=feuchte_tresh])
print("------ PM10 Stats cut ABOVE >  Feuchte Threshold {}% ------".format(feuchte_tresh))
ind_sensor_lookup(test_pm10Xfeuchte[test_feuchteXpm10>feuchte_tresh])
print("############################################")
print("------ PM10 Stats cut BELOW <= Feuchte Threshold {}% AND <= Temp Threshold {}°C------".format(feuchte_tresh, temp_thresh))
ind_sensor_lookup(test_pm10Xfeuchte[below_mask])
print("------ PM10 Stats cut ABOVE > Feuchte Threshold {}% AND > Temp Threshold {}°C------".format(feuchte_tresh, temp_thresh))
ind_sensor_lookup(test_pm10Xfeuchte[above_mask])

------ PM10 Stats cut BELOW <= Temp Threshold 3°C ------
Sensors that measured more than 70% of the Time according to the Conditional Remove = 0 from 115
------ PM10 Stats cut ABOVE >  Temp Threshold 3°C ------
Sensors that measured more than 70% of the Time according to the Conditional Remove = 50 from 115
############################################
------ PM10 Stats cut BELOW <= Feuchte Threshold 80%------
Sensors that measured more than 70% of the Time according to the Conditional Remove = 10 from 114
------ PM10 Stats cut ABOVE >  Feuchte Threshold 80% ------
Sensors that measured more than 70% of the Time according to the Conditional Remove = 37 from 114
############################################
------ PM10 Stats cut BELOW <= Feuchte Threshold 80% AND <= Temp Threshold 3°C------
Sensors that measured more than 70% of the Time according to the Conditional Remove = 0 from 114
------ PM10 Stats cut ABOVE > Feuchte Threshold 80% AND > Temp Threshold 3°C------
Sensors that measured

In [889]:
pm10Xtemp.to_csv("ressources/intersections/pm10Xtemp.cvs")
tempXpm10.to_csv("ressources/intersections/tempXpm10.cvs")

pm10Xfeuchte.to_csv("ressources/intersections/pm10Xfeuchte.cvs")
feuchteXpm10.to_csv("ressources/intersections/feuchteXpm10.cvs")

pm10XfeuchteXtemp.to_csv("ressources/intersections/pm10XfeuchteXtemp.cvs")
feuchteXtempXpm10.to_csv("ressources/intersections/feuchteXtempXpm10.cvs")
tempXfeuchteXpm10.to_csv("ressources/intersections/tempXfeuchteXpm10.cvs")